In [ ]:
import numpy as np
import scipy.stats as st
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_context('talk')

from matplotlib.animation import FuncAnimation

In [ ]:
#SENSITIVITY ANALYSIS
#Assume independence between variables so that first order sensitivities are enough

#set variable means x_i
#run simulation to obtain quantity of interest Q
#get the mean Q_0
#give nominated variances to variables - 1%? as var_i, sigma_i = sqrt(var_i)
#run simulation (num of variables) times with perturbed variables by h = eps*x_i
#get first order sensitivity (Q(x_i)-Q_0)/h
#obtain vector of sensitivities dQ_dx
#Calculate covariance matrix (only diagonal terms here) Sigma
#Obtain variance in Q via var_Q = dQ_dx.T @ Sigma @ dQ_dx
#Obtain uncertainty estimate via sigma_Q = sqrt(var_Q)

In [ ]:
#VARIABLES

#temp
#rad
#thick
#rr_coef
#dif_coef
#init_c
#max_c
#vol_per
#iapp

In [ ]:
#Set mean of variables here
#as well as initial dataframe
#mu should be a 9x1 array
params = ['temp','rad','thick','rr_coef','dif_coef','init_c','max_c','vol_per','iapp']
mu = ...
volts_df = pd.DataFrame(dict(parameter=params,mu=mu))

#Auxilliary 2D array of variable means, identical columns
Mu_s = np.tile(mu.T,(1,1000))

#Set value of perturbation here
#h_i = eps*mu_i
#Set h as a matrix for element-wise division, identical columns with elements = h_i
eps = 1e-6
h_i = eps*mu
h = np.tile(h_i.T,(1,1000))

#Obtain mean voltage curve V_0 here, each row identical
#9 rows representing variables
#1000 columns representing timesteps
V_0 = np.zeros((9,1000))
V_0 = ...

#Obtain perturbed voltages here
#9 rows representing variables
#1000 columns representing timesteps
Vs = np.zeros((9,1000))
Vs = ...

#Compute dV/dx as a function of time
dV_dx = np.divide((Vs - V_0),h)

#Compute absolute scaled sensitivity using element-wise multiplication
V_first_sensitivities = np.abs(Mu_s*dV_dx)

#each column i of the matrix represents the sensitivities for each paramter at time t=i 

In [ ]:
#plot as animation
fig, ax = plt.figure()
ax.set_xticklabels(volts_df.parameter, rotation=45)
ax.set_ylabel('Absolute scaled sensitivity of $V$')
ax.set_xlabel('Parameter')

#time between frames
intervaltime = 0.5

#definition of animation
def animate(i):
    V_df = volts_df.copy()
    V_df['scaled_sensitivity'] = V_first_sensitivities[:,i]
    ax = V_df['scaled_sensitivity'].abs().plot(xticks=V_df.index, logy=True)

#plot first frame
V_df = volts_df.copy()
V_df['scaled_sensitivity'] = V_first_sensitivities[:,0]
ax = V_df['scaled_sensitivity'].abs().plot(xticks=V_df.index, logy=True)

#plot animation
animate_sensitivities = FuncAnimation(fig, animate,interval=intervaltime, frames=range(1,len(mu)), blit=True)
plt.show()